### Combining Everything from `workbook.ipynb`

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import linregress
import os

In [9]:
def fit_function(x, *params, E_c=100):
    """
    This function defines the mathmatecal function we want to fit the data to. In our case this is E = E_c * (I/I_c)^n.
    Parameters are I_c and n. E_c is set to the critical electric field criterion. 
    """
    I_c, n = params 
    return E_c * (x/I_c)**n

def power_fitting(xdata, ydata):
    """
    Fits the function defined in fit_function().

    arguments:
    ----------
    - xdata 
    - ydata
    - E_c - Electric Field Criterion for Critical Current

    returns:
    --------
    - I_c - critical current
    - n - exponent of (I/I_c)^n
    - popt - all parameters from scipy.optimize.curve_fit
    """

    # Initial Guesses
    guess = [30, 20]

    try:
        popt, pcov = curve_fit(fit_function, xdata, ydata, p0=guess, ) # can add maxfev=5000
    except RuntimeError as e:
        print('Fit did not converge:', e)
    print(popt)
    I_c = popt[0]
    n = popt[1]

    return I_c, n, popt

# Iterate through files in specified directory #
def Analyse_Directory_PowerLawFit(directory):
    
    # Building a dataframe (table), each row is the analysis of one file in the directory
    rows = []
    for filename in os.listdir(directory):
        try:
            file_string = os.path.join(directory, filename).replace('\\','/')   # example string: 'data/P1_B1_Idefault_Ndefault.mat'
            df = pd.read_csv(file_string, skiprows=11, names=['current_A','voltage_uV','time_s'], sep='\\s+')
        except:
            print(f"Error: Cannot load {file_string}.")
            continue
        print(file_string)
        # Extract metadata from filename
        filename_split = file_string.split("_")
        angle = float(filename_split[-2])
        field = float(filename_split[-4])

        df['Efield_uVm-1'] = df['voltage_uV']/(1.4e-2) # Creating a new column for Electric field

        # Linear Fitting to half the data
        half = round(len(df)*0.5) # Finds the row index halfway through the data (usually around I = 18A)
        slope, intercept = linregress(df[:half]['current_A'], df[:half]['Efield_uVm-1'])[0:2] 
        linear_yvalues = df['current_A']*slope + intercept # The y-values of the linear fit extrapolated to the whole range of data that we will subtract from our data. 
        
        # Power Law Fitting to Cleaned Data (linear fit subtracted)
        xdata = df['current_A']
        ydata_clean = df['Efield_uVm-1'] - linear_yvalues
        I_c, n, popt = power_fitting(xdata, ydata_clean)

        # Append the analysis of this file to the dataset
        row = [angle, field, I_c, n]
        rows.append(row)

    columns=['angle', 'field', 'I_c', 'n']
    df_directory = pd.DataFrame(rows, columns=columns)
    return df_directory

In [12]:
# df_Ic_0 = Analyse_Directory_PowerLawFit('data/theta0/')
# df_Ic_45 = Analyse_Directory_PowerLawFit('data/theta45/')
# df_Ic_90 = Analyse_Directory_PowerLawFit('data/theta90/')
# df_Ic_135 = Analyse_Directory_PowerLawFit('data/theta135/')
df_Ic_180 = Analyse_Directory_PowerLawFit('data/theta180/')

# df_all = pd.concat([df_Ic_0, df_Ic_45, df_Ic_90, df_Ic_135, df_Ic_180])
# df_all
df_Ic_180

data/theta180/Measurement_5_0.00_Field_180_Angle.csv
[71.34946074 22.22349284]
data/theta180/Measurement_5_0.02_Field_180_Angle.csv
[62.01485637 15.74514893]
data/theta180/Measurement_5_0.04_Field_180_Angle.csv
[52.47387782 13.88885418]
data/theta180/Measurement_5_0.06_Field_180_Angle.csv
[46.24534755 13.06049193]
data/theta180/Measurement_5_0.08_Field_180_Angle.csv
[41.96242101 12.02266464]
data/theta180/Measurement_5_0.10_Field_180_Angle.csv
[40.64911024 12.98151468]
data/theta180/Measurement_5_0.20_Field_180_Angle.csv
[32.50350149 11.72418117]
data/theta180/Measurement_5_0.30_Field_180_Angle.csv
[27.53620898 10.01894792]
data/theta180/Measurement_5_0.40_Field_180_Angle.csv
[26.48486298 11.08569706]
data/theta180/Measurement_5_0.50_Field_180_Angle.csv
Fit did not converge: Optimal parameters not found: Number of calls to function has reached maxfev = 600.


UnboundLocalError: cannot access local variable 'popt' where it is not associated with a value